In [ ]:
import cv2
import numpy as np
import cv2
import numpy as np
from utils.da_transform import load_image
import json

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def center_square(image):
    # 이미지 크기 확인
    height, width, _ = image.shape

    # 가로와 세로 중 짧은 길이 결정
    min_dim = min(height, width)

    # 정사각형으로 이미지 자르기
    start_x = (width - min_dim) // 2
    start_y = (height - min_dim) // 2
    end_x = start_x + min_dim
    end_y = start_y + min_dim
    cropped_image = image[start_y:end_y, start_x:end_x]

    return cropped_image

def inference_yw(image, session) -> list:
    height, width = image.shape[:2]
    if width != 640 and height == 640:
        raise Exception("이미지 크기를 640x640으로 맞춰주세요.")
    
    image = image.astype(np.float32) / 255.0
    image = np.transpose(image, (2, 0, 1))  # Change data layout from HWC to CHW
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    input_name = session.get_inputs()[0].name
    output_names = [o.name for o in session.get_outputs()]
    outputs = session.run(output_names, {input_name: image})

    class_ids = outputs[0][0]
    bbox = outputs[1][0]
    scores = outputs[2][0]
    additional_info = outputs[3][0]
    score_threshold = [0.03, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.01]

    metadata = []

    for i, score in enumerate(scores):
        if additional_info[i] >= 0:
            if score > score_threshold[additional_info[i]]:
                metadata.append(bbox[i].tolist() + [int(additional_info[i])])
    
    return metadata

def inference_da(image, session):
    image, (orig_h, orig_w) = load_image(image)
    depth = session.run(None, {"image": image})[0]
    depth = cv2.resize(depth[0, 0], (orig_w, orig_h))
    
    return depth

def update_figure(metadata, frame):
        is_danger = [False] * 9

        for bbox in metadata:
            x1, y1, x2, y2 = bbox[:4]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            class_id = bbox[4]
            median_point = bbox[5]
            max_point = bbox[6]                     
            mean_point = bbox[7] 
            middle_point = bbox[8]
            x = bbox[9]
            y = bbox[10]
            rad = bbox[11]
            distance = bbox[12]

            if class_id < 9 and distance < 5:
                is_danger[class_id] = True

        return is_danger

# 파일 경로 지정
input_file_path = "PATH/TO/JSON"

# JSON 파일 불러오기
with open(input_file_path, "r") as infile:
    dist_data = json.load(infile)

# GT가 있는 폴더 경로 (txt)
txt_path = 'PATH/TO/GT'

answer = [[], []] # 각각 GT에 따라 거리값 저장
for filename in dist_data.keys():
        infos = dist_data[filename]

        GT = [False] * len(infos)
        # 텍스트 파일 불러오기
        with open(txt_path + '/' + filename[:-4] + '.txt', "r") as file:
            for line in file:
                # 쉼표로 분리하여 데이터 추출
                data = line.strip().split(',')
                if len(data) < 3:
                    print('잘못된 GT: ', data)
                    continue
                for i in range(int(data[1]), int(data[2])):
                    GT[i] = True

        for i in range(1, len(infos)):
            dist = infos[i]['min_dist']
            if dist == 100:
                continue
            if GT[i]:
                answer[0].append(dist)
            else:
                answer[1].append(dist)

# 박스 플롯 그리기
plt.boxplot([answer[0], answer[1]], labels=['Danger', 'Not Danger'])

# 그래프 제목 설정
plt.title('Comparison')

# x축 레이블 설정
plt.xlabel('Cases')

# y축 레이블 설정
plt.ylabel('Distance (min)')

# 선 그리기
plt.axhline(y=5, color='r', linestyle='--', linewidth=2)  # y=5인 선
plt.axhline(y=10, color='b', linestyle='--', linewidth=2)  # y=10인 선

# 그래프 출력
plt.show()

# 바이올린 플롯 그리기
plt.violinplot(answer, showmeans=True)

# 선 그리기
plt.axhline(y=5, color='r', linestyle='--', linewidth=2)  # y=5인 선
plt.axhline(y=10, color='b', linestyle='--', linewidth=2)  # y=10인 선

# 그래프 출력
plt.show()
